In [45]:
import os
import cv2
import numpy as np

## Gloabl Variables and Paths

In [46]:
# paths
THIS_PATH = os.getcwd()
HOME_PATH =  os.path.expanduser('~')
DEADLIFT_VIDEO_PATH = os.path.join(HOME_PATH, 'Desktop', 'QHacks', 'raw_deadlift_videos')
BAD_DEADLIFT_IMAGE_PATH = os.path.join(HOME_PATH, 'Desktop', 'QHacks', 'bad_deadlift_images')
GOOD_DEADLIFT_IMAGE_PATH = os.path.join(HOME_PATH, 'Desktop', 'QHacks', 'good_deadlift_images')

## Convert Videos into Images

In [47]:
# get all videos in directory for each class
all_deadlift_videos = os.listdir(DEADLIFT_VIDEO_PATH)
all_deadlift_videos = [video for video in all_deadlift_videos if '.mp4' in video]
all_deadlift_videos

['BadDeadlift11.mp4',
 'BadDeadlift10.mp4',
 'BadDeadlift9.mp4',
 'BadDeadlift8.mp4',
 'Deadlifts4.mp4',
 'Deadlift14.mp4',
 'Deadlift9.mp4',
 'Deadlift10.mp4',
 'Deadlifts3.mp4',
 'Deadlift8.mp4',
 'Deadlift13.mp4',
 'Deadlifts1.mp4',
 'Deadlift12.mp4',
 'Deadlift6.mp4',
 'Deadlift7.mp4',
 'Deadlift5.mp4',
 'BadDeadlift5.mp4',
 'BadDeadlift4.mp4',
 'BadDeadlift6.mp4',
 'BadDeadlift7.mp4',
 'BadDeadlift3.mp4',
 'BadDeadlift2.mp4',
 'BadDeadlift1.mp4']

In [75]:
def get_frames(video_path, location, count, sec):
    video_name = video_path.split('/')[-1]
    filename = video_name.split('.')[0]
    
    vidcap = cv2.VideoCapture(video_path)
    vidcap.set(cv2.CAP_PROP_POS_MSEC, sec*1000)
    hasFrames,image = vidcap.read()
    if hasFrames:
        image_name = filename + "_" + str(count) + ".jpg"
        path = os.path.join(location, image_name)
        cv2.imwrite(path, image) 

    return hasFrames

In [76]:
def convert_video_to_images(video_path, location, frame_rate):
    sec = 0
    count = 1

    # first image
    success = get_frames(video_path, location, count, sec)
    # rest of the images
    while success:
        count = count + 1
        sec = sec + frame_rate
        sec = np.round(sec, 2)
        success = get_frames(video_path, location, count, sec)

    return None

In [78]:
for video in all_deadlift_videos:
    video_path = os.path.join(DEADLIFT_VIDEO_PATH, video)
    if 'Bad' in video:
        location = BAD_DEADLIFT_IMAGE_PATH
    else:
        location = GOOD_DEADLIFT_IMAGE_PATH

    convert_video_to_images(video_path=video_path, location=location, frame_rate=0.1) # 0.033 is 60 imgs/sec